# Text Mining Project: AUC Courses Recommender
## Desrciption
In this notebook you will find the source code for the Amsterdam University College (AUC) Course Recommender which was made for a Project for the course called Text Mining. (Dalszy opis ...)

## Code
Imports:

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.parsing.preprocessing import remove_stopwords

Loading and dealing with the dataset

In [11]:
#Loading the data in as a DataFrame.
data = pd.read_csv("datasets/recommender_dataset.csv")

#Droping the rows which had noting in the row in the course_catalogue_number column.
courses = data.dropna(subset=['course_catalogue_number'])

#Getting out all of the duplicates form the data, just in case there are any.
indices = pd.Series(courses.index, index=courses['course_catalogue_number']).drop_duplicates()

In [13]:
print(courses['is_part_of'][1].lower())

master's cultural and social anthropology


In [43]:
#Removing the stop words from columns: "is_part_of" and "course_description"
stop = set(stopwords.words('english'))
for i in range(len(courses.count())):
    courses.loc[i, 'course_name'] = courses.loc[i, 'course_name'].lower()
    courses.loc[i, 'college_graduate'] = courses.loc[i, 'college_graduate'].lower()
    courses.loc[i, 'language_of_instruction'] = courses.loc[i, 'language_of_instruction'].lower()
    courses.loc[i, 'is_part_of'] = courses.loc[i, 'is_part_of'].lower()
    courses.loc[i, 'is_part_of'] = remove_stopwords(courses.loc[i, 'is_part_of'].lower())
    courses.loc[i, 'course_description'] = courses.loc[i, 'course_description'].lower()
    courses.loc[i, 'course_description'] = remove_stopwords(courses.loc[i, 'course_description'].lower())

In [45]:
print(courses['language_of_instruction'][0])

english


In [49]:
#Making a new column with all of the text from the row.
courses['combined_text'] = courses[['course_name', 'is_part_of', 'college_graduate', 'course_description']].fillna('').agg(' '.join, axis=1)

/var/folders/nz/8r1yxrw92259wsg5lql4t6hm0000gn/T/ipykernel_65388/3429331287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  courses['combined_text'] = courses[['course_name', 'is_part_of', 'college_graduate', 'course_description']].fillna('').agg(' '.join, axis=1)


In [27]:
#Vectorizing the column courses['combined_text'].
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(courses['combined_text'])

In [29]:
#Getting the cosine similarities between the rows of the matrix (these are besically the similiarities between the courses).
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [31]:
#Test of the cosine similarities to make the first predictions and see if they 
#are somewhat correct.
def recommend_courses(course_id, cosine_sim=cosine_sim, top_n=5):
    idx = indices.get(course_id)
    if idx is None:
        return []
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1] 
    course_indices = [i[0] for i in sim_scores]
    return courses.iloc[course_indices][['course_name', 'course_catalogue_number']]

In [33]:
recommend_courses('118110146Y')

,course_name,course_catalogue_number
1999,Light on Matter,5113LIOM6Y
2304,Materials for Light-Energy Conversion,5354MFLE6Y
2444,Molecular Spectroscopy,5112MOSS6Y
1890,Introduction to the Energy Transition,900109SCIY
2673,Physics of Sustainable Energy,5092PHSE6Y


In [35]:
recommend_courses('900264SCIY')

,course_name,course_catalogue_number
477,Business Law and Ethics,6012B0428Y
785,Corporate Law,6012B0458Y
1738,International Law of Military Operations,3854INQ8VY
188,Amsterdam Law Clinics,3554ALCLKY
2783,Principles and Foundations of International Law,3554PRFIVY
